In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Hung\\Desktop\\ML-Ops\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Hung\\Desktop\\ML-Ops'

In [8]:
from dataclasses import dataclass
from pathlib import Path

# class to save configurations for data ingestion
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [12]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

# class to create dirs for the raw data
class DataTransformationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )
        
        return data_transformation_config

In [10]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config
        
    def train_test_split(self) -> bool:
        data = pd.read_csv(self.config.data_path)
        
        train, test = train_test_split(data)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)
        
        logger.info("Train-Test-Split was successful!")
        logger.info(f"Train-shape{train.shape}")
        logger.info(f"Test-shape{test.shape}")

In [13]:
try:
    config = DataTransformationManager()
    data_validation_config = config.get_data_transformation_config()
    data_validation = DataTransformation(config=data_validation_config)
    data_validation.train_test_split()
except Exception as e:
    raise e

[2023-10-14 17:54:10,116: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-14 17:54:10,117: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-14 17:54:10,120: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-14 17:54:10,121: INFO: common: created directory at: artifacts]
[2023-10-14 17:54:10,128: INFO: common: created directory at: artifacts/data_transformation]
[2023-10-14 17:54:10,150: INFO: 1231719952: Train-Test-Split was successful!]
[2023-10-14 17:54:10,151: INFO: 1231719952: Train-shape(1199, 12)]
[2023-10-14 17:54:10,152: INFO: 1231719952: Test-shape(400, 12)]
